In [1]:
!pip install lightfm
!pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=879186 sha256=58a34b3adf325f3ab2241e5927f4c661585f8dc8d5dcd6264fa95000ea873bd3
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 43.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

# Функции из 1-ого вебинара
import os, sys, itertools, copy

sys.path.append('/content/drive/MyDrive/Rec_systems')

# module_path = os.path.abspath(os.path.join(os.pardir))
# if module_path not in sys.path:
#     sys.path.append(module_path)

# from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items

/usr/local/lib/python3.10/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [3]:
data = pd.read_csv('/content/drive/MyDrive/Rec_systems/retail_train.csv')
item_features = pd.read_csv('/content/drive/MyDrive/Rec_systems/product.csv')
user_features = pd.read_csv('/content/drive/MyDrive/Rec_systems/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train, 5000, item_features)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

/content/drive/MyDrive/Rec_systems/src/utils.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


Decreased # items from 86865 to 5001


In [5]:
user_item_matrix = pd.pivot_table(data_train,
                                  index='user_id', columns='item_id',
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count',
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

item_id,117847,818981,819255,819308,819400,819487,819590,819594,819840,819845,...,15926775,15926844,15926886,15972074,15972298,15972565,15972790,16100266,16729299,16729415
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]
test_user_item_matrix = pd.pivot_table(data_test,
                                  index='user_id', columns='item_id',
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count',
                                  fill_value=0
                                 )

test_user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

In [7]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [8]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)

item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

In [9]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [10]:
def learning_curve(model, train, test, user_item_matrix, user_feat_lightfm, item_feat_lightfm, k=5):

    model.fit(train,
              sample_weight=coo_matrix(user_item_matrix),
              user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
              item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
              epochs=15,
              num_threads=8)

    train_precision = precision_at_k(model,
                                    train,
                                    user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                    item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                    k=5).mean()

    return model, train_precision

In [17]:
def grid_search_learning_curve(base_model, train, test, user_item_matrix, user_feat_lightfm, item_feat_lightfm, param_grid, patk=5):
    curves = []
    keys, values = zip(*param_grid.items())
    for v in itertools.product(*values):
        params = dict(zip(keys, v))
        this_model = copy.deepcopy(base_model)
        print_line = []
        for k, v in params.items():
            setattr(this_model, k, v)
            print_line.append((k, v))

        print(' | '.join('{}: {}'.format(k, v) for (k, v) in print_line))
        _, test_patk  = learning_curve(this_model, train, test, user_item_matrix, user_feat_lightfm, item_feat_lightfm, k=patk)

        curves.append({'params': params,
                       'patk': {'train': test_patk}})
        print(f'MAP@5={test_patk}')
    return curves

In [15]:
def print_log(row, header=False, spacing=12):
    top = ''
    middle = ''
    bottom = ''
    for r in row:
        top += '+{}'.format('-'*spacing)
        if isinstance(r, str):
            middle += '| {0:^{1}} '.format(r, spacing-2)
        elif isinstance(r, int):
            middle += '| {0:^{1}} '.format(r, spacing-2)
        elif isinstance(r, float):
            middle += '| {0:^{1}.5f} '.format(r, spacing-2)
        bottom += '+{}'.format('='*spacing)
    top += '+'
    middle += '|'
    bottom += '+'
    if header:
        print(top)
        print(middle)
        print(bottom)
    else:
        print(middle)
        print(top)

In [19]:
param_grid = {'no_components': [30],
              'loss':['bpr', 'warp'], # 'warp'
              'learning_rate':[5e-2, 1e-2, 1e-3],
              'item_alpha':[0.1, 0.01],
              'user_alpha':[0.1, 0.01],
              }

base_model = LightFM()
curves = grid_search_learning_curve(base_model,
                                    train=sparse_user_item,
                                    test=data_test,
                                    user_item_matrix=user_item_matrix,
                                    user_feat_lightfm=user_feat_lightfm,
                                    item_feat_lightfm=item_feat_lightfm,
                                    param_grid=param_grid,
                                    patk=5)

no_components: 30 | loss: bpr | learning_rate: 0.05 | item_alpha: 0.1 | user_alpha: 0.1
MAP@5=0.20937126874923706
no_components: 30 | loss: bpr | learning_rate: 0.05 | item_alpha: 0.1 | user_alpha: 0.01
MAP@5=0.1996796429157257
no_components: 30 | loss: bpr | learning_rate: 0.05 | item_alpha: 0.01 | user_alpha: 0.1
MAP@5=0.14144974946975708
no_components: 30 | loss: bpr | learning_rate: 0.05 | item_alpha: 0.01 | user_alpha: 0.01
MAP@5=0.4385262429714203
no_components: 30 | loss: bpr | learning_rate: 0.01 | item_alpha: 0.1 | user_alpha: 0.1
MAP@5=0.4385262429714203
no_components: 30 | loss: bpr | learning_rate: 0.01 | item_alpha: 0.1 | user_alpha: 0.01
MAP@5=0.4385262429714203
no_components: 30 | loss: bpr | learning_rate: 0.01 | item_alpha: 0.01 | user_alpha: 0.1
MAP@5=0.3485783338546753
no_components: 30 | loss: bpr | learning_rate: 0.01 | item_alpha: 0.01 | user_alpha: 0.01
MAP@5=0.3498598635196686
no_components: 30 | loss: bpr | learning_rate: 0.001 | item_alpha: 0.1 | user_alpha: 0